In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
homes= pd.read_csv('../input/train.csv')

In [ ]:
homes.tail()

In [ ]:
homes_test= pd.read_csv('../input/test.csv')

In [ ]:
homes_test.tail()

In [ ]:
# Combine train and test for cleanup
homes_all = homes.drop(columns=['SalePrice']).append(homes_test)

In [ ]:
homes_all.shape

In [ ]:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self, categoricals):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
        self.categories = categoricals
        
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if c in self.categories else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [ ]:
continuous_features = [
    'LotArea', 
    'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', '2ndFlrSF', 'LowQualFinSF',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold'

]



In [ ]:
categorical_features = [
    'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition',
    
    'OverallQual', 'OverallCond'
]



In [ ]:
ignore_features = [
'BsmtFinSF1'
, '1stFlrSF'
, 'YrSold'
, 'YearBuilt'
, 'YearRemodAdd'
, 'GarageYrBlt'
, 'GrLivArea'
]


In [ ]:
homes_all=homes_all.drop(ignore_features, axis=1)

In [ ]:
# Impute
homes_all = DataFrameImputer(categorical_features).fit_transform(homes_all)

# One hot encoding
homes_all = pd.get_dummies(homes_all,columns=categorical_features,drop_first=True)

In [ ]:
homes_all.shape

In [ ]:
# Split it back into home and home_test
homes2 = homes_all[homes_all.Id <= 1460]
homes2['SalePrice'] = homes['SalePrice']

In [ ]:
homes2[['Id', 'SalePrice']].tail()

In [ ]:
homes[['Id', 'SalePrice']].tail()

In [ ]:
homes = homes2
homes_test = homes_all[homes_all.Id > 1460]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
sns.distplot(homes.SalePrice)

In [ ]:
plt.hist(homes['SalePrice'])

In [ ]:
sns.distplot(homes.SalePrice)

In [ ]:
#pd.isna(homes).sum()

In [ ]:
pd.isna(homes.SalePrice).sum()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import KFold

In [ ]:
Upper_whisker=((homes.SalePrice.quantile(0.75)-homes.SalePrice.quantile(0.25))*1.5)+homes.SalePrice.quantile(0.75)
homes2=homes[homes.SalePrice<Upper_whisker]

In [ ]:
X = homes2[continuous_features]
y = homes2['SalePrice']

In [ ]:
# instantiate and fit
linreg = LinearRegression()
linreg.fit(X, y)

In [ ]:
y_pred = linreg.predict(X)
metrics.r2_score(y, y_pred)

In [ ]:
np.sqrt(metrics.mean_squared_error(y, y_pred))

In [ ]:
y.mean()

In [ ]:
X.corr()

In [ ]:
np.any(np.isnan(X))

In [ ]:
np.all(np.isfinite(X))

In [ ]:
np.any(np.isnan(y))

In [ ]:
scores = cross_val_score(linreg, X, y, cv=10, scoring='neg_mean_squared_error')
print (scores)

In [ ]:
# fix the sign of MSE scores
mse_scores = -scores
print (mse_scores)

In [ ]:
# convert from MSE to RMSE
rmse_scores = np.sqrt(mse_scores)
print (rmse_scores)

In [ ]:
# calculate the average RMSE
print (rmse_scores.mean())

In [ ]:
sns.boxplot(y)

In [ ]:
y.describe()

In [ ]:
# homes3=pd.get_dummies(homes2,columns=categorical_features,drop_first=True)

In [ ]:
# homes4=homes2.drop(['Id','SalePrice','LotFrontage','MasVnrArea','GarageYrBlt'],axis=1)
homes4=homes2.drop(['Id','SalePrice'],axis=1)

In [ ]:
X = homes4
y = homes2['SalePrice']

In [ ]:
# instantiate and fit
linreg = LinearRegression()
linreg.fit(X, y)

In [ ]:
y_pred = linreg.predict(X)
metrics.r2_score(y, y_pred)

In [ ]:
np.sqrt(metrics.mean_squared_error(y, y_pred))

In [ ]:
def build_model(df, feature_cols, target_col, folds,log=False):
    X = df[feature_cols]
    if log:
        y = np.log(df[target_col])
    else:
        y = df[target_col]

                      
    
    # instantiate and fit
    linreg = LinearRegression()
    
    scores = cross_val_score(linreg, X, y, cv=folds, scoring='neg_mean_squared_error')
    print ("Scores : %s"%scores)

    # fix the sign of MSE scores
    mse_scores = -scores
    print ("MSEs : %s"%mse_scores)

           
    # convert from MSE to RMSE
    rmse_scores = np.sqrt(mse_scores)
    print ("RMSEs : %s"%rmse_scores)

    # calculate the average RMSE
    print ("RMSE = %f"%rmse_scores.mean())
    linreg.fit(X,y)
    return linreg


In [ ]:
model=build_model(homes2,homes4.columns,['SalePrice'],5, log=True)

In [ ]:
# homes_test= pd.read_csv('../input/test.csv')

In [ ]:
# homes_test = DataFrameImputer(categorical_features).fit_transform(homes_test)
#imputer = DataFrameImputer(categorical_features)
#imputer = imputer.fit(X)
#homes_test = imputer.transform(homes_test)
#homes_test.columns[homes_test.isna().any()].tolist()

In [ ]:
homes_test.head()

In [ ]:
#X_test=pd.get_dummies(homes_test,columns=categorical_features,drop_first=True).drop(['Id','LotFrontage','MasVnrArea','GarageYrBlt'],axis=1)

# X_test = homes_test.drop(['Id','LotFrontage','MasVnrArea','GarageYrBlt'], axis = 1)
X_test = homes_test.drop(['Id'], axis = 1)

In [ ]:
missing_columns = set(X.columns)-set(X_test.columns)
missing_columns

In [ ]:
print(", ".join(X.columns))

In [ ]:
print(", ".join(X_test.columns))

In [ ]:
print(X.shape)
print(X_test.shape)

In [ ]:
y_test_pred=model.predict(X_test)

In [ ]:
y_test_pred

In [ ]:
np.exp(y_test_pred).mean()

In [ ]:
y.mean()

In [ ]:
print(y_test_pred)

In [ ]:

homes_test['SalePrice'] = y_test_pred[:,0]

In [ ]:
homes_test['SalePrice'] = np.exp(homes_test['SalePrice'])
homes_test[['Id', 'SalePrice']]

In [ ]:
homes_test['SalePrice'].describe()

In [ ]:
homes.SalePrice.describe()

In [ ]:
print(os.listdir("."))

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor    

def calculate_vif_(X, thresh=5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]

In [ ]:
#vif=calculate_vif_(homes_all.drop(['Id'],axis=1))

In [ ]:
homes_test[['Id', 'SalePrice']].to_csv('submission.csv', index = False)